# Importing Dependencies

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input,LSTM,Dense


/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


# Loading data

In [2]:
dataset_path='/kaggle/input/french-eng/fra.txt'

In [3]:
input_text=[]
target_text=[]
input_char=set()
target_char=set()

In [4]:
with open(dataset_path,'r',encoding='utf-8') as f:
    lines=f.read().split('\n')

In [5]:
num_sample=10000

# Text Preprocessing

Splitting into input and target sequences, processing the target sequences for Seq2Seq compatibility, storing the sequences in separate lists, and creating character sets for both input and target texts by collecting unique characters.

In [6]:
for line in lines[:min(num_sample,len(lines)-1)]:
    input_txt,target_txt,_=line.split('\t')
    target_txt='\t'+target_txt+'\n'
    input_text.append(input_txt)
    target_text.append(target_txt)
    for char in input_txt:
        if char not in input_char:
            input_char.add(char)
    for char in target_txt:
        if char not in target_char:
            target_char.add(char)

The maximum sequence lengths determine the input and output lengths the model can handle, while the counts of unique tokens form the basis for the model's vocabulary size

In [7]:
input_char=sorted(list(input_char))
target_char=sorted(list(target_char))
num_encoder_token=len(input_char)
num_decoder_token=len(target_char)
max_encoder_seq=max([len(txt) for txt in input_text])
max_decoder_seq=max([len(txt) for txt in target_text])

In [8]:
num_encoder_token,num_decoder_token,max_encoder_seq,max_decoder_seq

(70, 93, 14, 59)

The dictionaries provide a way to convert characters in the input and target sequences into their corresponding indices in the model's vocabulary. This mapping facilitates the one-hot encoding of the sequences, allowing the model to understand and process the textual data in a numerical format

In [9]:
input_token_index=dict([(char,i) for i, char in enumerate(input_char)])
target_token_index=dict([(char,i) for i, char in enumerate(target_char)])

In [10]:
input_token_index,target_token_index

({' ': 0,
  '!': 1,
  '"': 2,
  '$': 3,
  '%': 4,
  '&': 5,
  "'": 6,
  ',': 7,
  '-': 8,
  '.': 9,
  '0': 10,
  '1': 11,
  '2': 12,
  '3': 13,
  '5': 14,
  '7': 15,
  '8': 16,
  '9': 17,
  ':': 18,
  '?': 19,
  'A': 20,
  'B': 21,
  'C': 22,
  'D': 23,
  'E': 24,
  'F': 25,
  'G': 26,
  'H': 27,
  'I': 28,
  'J': 29,
  'K': 30,
  'L': 31,
  'M': 32,
  'N': 33,
  'O': 34,
  'P': 35,
  'Q': 36,
  'R': 37,
  'S': 38,
  'T': 39,
  'U': 40,
  'V': 41,
  'W': 42,
  'Y': 43,
  'a': 44,
  'b': 45,
  'c': 46,
  'd': 47,
  'e': 48,
  'f': 49,
  'g': 50,
  'h': 51,
  'i': 52,
  'j': 53,
  'k': 54,
  'l': 55,
  'm': 56,
  'n': 57,
  'o': 58,
  'p': 59,
  'q': 60,
  'r': 61,
  's': 62,
  't': 63,
  'u': 64,
  'v': 65,
  'w': 66,
  'x': 67,
  'y': 68,
  'z': 69},
 {'\t': 0,
  '\n': 1,
  ' ': 2,
  '!': 3,
  '%': 4,
  '&': 5,
  "'": 6,
  '(': 7,
  ')': 8,
  ',': 9,
  '-': 10,
  '.': 11,
  '0': 12,
  '1': 13,
  '2': 14,
  '3': 15,
  '5': 16,
  '8': 17,
  '9': 18,
  ':': 19,
  '?': 20,
  'A': 21,
  'B'

Arrays are preallocated with zeros to accommodate the one-hot encoded representations of the input sequences, target sequences, and their corresponding token vocabularies, which will be filled in during the data encoding process

In [11]:
encoder_input = np.zeros((len(input_text), max_encoder_seq, num_encoder_token), dtype="float32")
decoder_input = np.zeros((len(target_text), max_decoder_seq, num_decoder_token), dtype="float32")
decoder_target = np.zeros((len(target_text), max_decoder_seq, num_decoder_token), dtype="float32")

One-hot encoded representations of the input and target sequences, crucial for training sequence-to-sequence models. The encoding is based on the character indices obtained from the dictionaries input_token_index and target_token_index

In [12]:
for i, (input_txt, target_txt) in enumerate(zip(input_text, target_text)):
    for t, char in enumerate(input_txt):
        encoder_input[i, t, input_token_index[char]] = 1.0
    encoder_input[i, t + 1 :, input_token_index[" "]] = 1.0
    
    for t, char in enumerate(target_txt):
        decoder_input[i, t, target_token_index[char]] = 1.0
        if t > 0:
            decoder_target[i, t - 1, target_token_index[char]] = 1.0
    decoder_input[i, t + 1 :, target_token_index[" "]] = 1.0
    decoder_target[i, t:, target_token_index[" "]] = 1.0




In [13]:
encoder_input[0].shape

(14, 70)

In [14]:
latent_dim=256

# Model Building

Encoder part of a sequence-to-sequence model, where input sequences are processed by an LSTM layer to produce output sequences and internal states that are utilized by the decoder during the decoding phase

In [15]:
encoder_inputs=Input(shape=(None,num_encoder_token))
encoder=LSTM(latent_dim,return_state=True)
encoder_outputs,state_h,state_c=encoder(encoder_inputs)
encoder_states=[state_h,state_c]

Decoder part of the sequence-to-sequence model, allowing it to process input sequences and generate output sequences using an LSTM layer followed by a dense layer with a softmax activation for token prediction

In [16]:
decoder_inputs=Input(shape=(None,num_decoder_token))
decoder_lstm=LSTM(latent_dim,return_sequences=True,return_state=True)
decoder_outputs,_,_=decoder_lstm(decoder_inputs,initial_state=encoder_states)
decoder_dense=Dense(num_decoder_token,activation='softmax')
decoder_outputs=decoder_dense(decoder_outputs)

In [17]:
batch_size=64
epochs=100

In [18]:
model=Model([encoder_inputs,decoder_inputs],decoder_outputs)
model.compile(optimizer='rmsprop',loss='categorical_crossentropy',metrics=['accuracy'])


# Model Training

In [19]:
model.fit([encoder_input,decoder_input],decoder_target,batch_size=batch_size,epochs=epochs,validation_split=0.2)

Epoch 1/100
125/125 [==============================] - 9s 21ms/step - loss: 1.2152 - accuracy: 0.7329 - val_loss: 1.0608 - val_accuracy: 0.7104
Epoch 2/100
125/125 [==============================] - 2s 12ms/step - loss: 0.9364 - accuracy: 0.7485 - val_loss: 0.9714 - val_accuracy: 0.7201
Epoch 3/100
125/125 [==============================] - 2s 12ms/step - loss: 0.8385 - accuracy: 0.7684 - val_loss: 0.8773 - val_accuracy: 0.7496
Epoch 4/100
125/125 [==============================] - 2s 12ms/step - loss: 0.7397 - accuracy: 0.7918 - val_loss: 0.7750 - val_accuracy: 0.7780
Epoch 5/100
125/125 [==============================] - 2s 12ms/step - loss: 0.6662 - accuracy: 0.8076 - val_loss: 0.7113 - val_accuracy: 0.7953
Epoch 6/100
125/125 [==============================] - 2s 12ms/step - loss: 0.6342 - accuracy: 0.8160 - val_loss: 0.7002 - val_accuracy: 0.8009
Epoch 7/100
125/125 [==============================] - 2s 12ms/step - loss: 0.5987 - accuracy: 0.8247 - val_loss: 0.6579 - val_accuracy:

# Sequence-to-Sequence Model Inference Setup

Preparing the encoder and decoder models, setting up the decoding function, and creating dictionaries to facilitate the conversion between indices and characters for inference in a sequence-to-sequence model

In [20]:
encoder_model=Model(encoder_inputs,encoder_states)
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

reverse_input_char_index=dict((i,char) for char,i in input_token_index.items())
reverse_target_char_index=dict((i,char) for char,i in target_token_index.items())

def decode_sequence(input_seq):
    states_value = encoder_model.predict(input_seq)
    target_seq = np.zeros((1, 1, num_decoder_token))
    target_seq[0, 0, target_token_index['\t']] = 1.
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq):
            stop_condition = True
        target_seq = np.zeros((1, 1, num_decoder_token))
        target_seq[0, 0, sampled_token_index] = 1.
        states_value = [h, c]
        
    return decoded_sentence



# Inference Results for Sequence-to-Sequence Model on Sample Data 

Performing inference on a subset of sequences, printing the original input sentences, and displaying the corresponding output sentences generated by the sequence-to-sequence model.

In [21]:
for seq_index in range(100):
    input_seq=encoder_input[seq_index:seq_index+1]
    decoded_sentence=decode_sequence(input_seq)
    print("____")
    print('Input sentence :',input_text[seq_index])
    print('Output sentence :',decoded_sentence)


1/1 [==============================] - 0s 19ms/step
____
Input sentence : Go.
Output sentence : Cassez-vous !

1/1 [==============================] - 0s 20ms/step
____
Input sentence : Go.
Output sentence : Cassez-vous !

1/1 [==============================] - 0s 20ms/step
____
Input sentence : Go.
Output sentence : Cassez-vous !

1/1 [==============================] - 0s 20ms/step
____
Input sentence : Go.
Output sentence : Cassez-vous !

1/1 [==============================] - 0s 19ms/step
____
Input sentence : Hi.
Output sentence : Cela tout-vour.

1/1 [==============================] - 0s 19ms/step
____
Input sentence : Hi.
Output sentence : Cela tout-vour.

1/1 [==============================] - 0s 19ms/step
____
Input sentence : Run!
Output sentence : File !

1/1 [==============================] - 0s 20ms/step
____
Input sentence : Run!
Output sentence : File !

1/1 [==============================] - 0s 20ms/step
____
Input sentence : Run!
Output sentence : File !

1/1 [==========